In [8]:
from pystoi import stoi
from pesq import pesq
import numpy as np
import soundfile as sf
import os

In [13]:
def measure_audio_quality(clean_file, noisy_file, sample_rate=16000):
    """
    음성 파일의 STOI와 PESQ 품질을 측정하는 함수
    
    Parameters:
    - clean_file: 원본 깨끗한 음성 파일 경로
    - noisy_file: 노이즈가 추가된 음성 파일 경로
    - sample_rate: 오디오 샘플링 레이트 (기본값 16000 Hz)
    
    Returns:
    - Dictionary containing STOI and PESQ scores
    """
    # 오디오 파일 로드
    clean_data, _ = sf.read(clean_file)
    if len(clean_data.shape) > 1:
        clean_data = clean_data[:, 0]
    noisy_data, _ = sf.read(noisy_file)
    if len(noisy_data.shape) > 1:
        noisy_data = noisy_data[:, 0]
    
    # STOI 측정
    # STOI는 0-1 사이의 값, 1에 가까울수록 음성 명료도 높음
    stoi_score = stoi(clean_data, noisy_data, sample_rate)
    
    # PESQ 측정
    # PESQ는 -0.5에서 4.5 사이의 값, 4.5에 가까울수록 음성 품질 우수
    # 'wb' 모드는 광대역(wide-band) 음성에 적합
    pesq_score = pesq(sample_rate, clean_data, noisy_data, 'wb')
    
    return {
        'STOI': stoi_score,
        'PESQ': pesq_score
    }

In [17]:
from si_sdr import si_sdr

def measure_si_sdr(clean_file, noisy_file):
    # 오디오 파일 로드
    clean_data, _ = sf.read(clean_file)
    if len(clean_data.shape) > 1:
        clean_data = clean_data[:, 0]
    if clean_data.dtype == np.int16:
        clean_data = (clean_data / 32768.0).astype(np.float64)
    noisy_data, _ = sf.read(noisy_file)
    if len(noisy_data.shape) > 1:
        noisy_data = noisy_data[:, 0]
    if noisy_data.dtype == np.int16:
        noisy_data = (noisy_data / 32768.0).astype(np.float64)
    si_sdr_score = si_sdr(clean_data, noisy_data)
    return si_sdr_score


In [15]:


if __name__ == "__main__":
    SIG_DIR = "sounds/before_noise"
    NOISY_DIR = "sounds/after_noise"
    DENOISED_DIR = "sounds/after_noise_denoised"
    
    for file in os.listdir(NOISY_DIR):
        if not file.endswith(".wav"):
            continue
        clean_file = os.path.join(SIG_DIR, f'{file.split("_")[0]}.wav')
        noisy_file = os.path.join(NOISY_DIR, file)
        denoised_file = os.path.join(DENOISED_DIR, file)
        quality_scores = measure_audio_quality(clean_file, noisy_file)
        denoised_quality_scores = measure_audio_quality(clean_file, denoised_file)
        print(f"노이즈제거 전: {file}, STOI: {quality_scores['STOI']:.4f}, PESQ: {quality_scores['PESQ']:.4f}")
        print(f"노이즈제거 후: {file}, STOI: {denoised_quality_scores['STOI']:.4f}, PESQ: {denoised_quality_scores['PESQ']:.4f}")

노이즈제거 전: speech4_단열폼 미장 소음.wav, STOI: 0.6045, PESQ: 1.0917
노이즈제거 후: speech4_단열폼 미장 소음.wav, STOI: 0.5415, PESQ: 1.1236
노이즈제거 전: speech4_대형 환기팬 및 설비 소음.wav, STOI: 0.7086, PESQ: 1.1352
노이즈제거 후: speech4_대형 환기팬 및 설비 소음.wav, STOI: 0.6922, PESQ: 1.2234
노이즈제거 전: speech3_보온 작업 근접 소음 및 선박 망치질.wav, STOI: 0.6944, PESQ: 1.0778
노이즈제거 후: speech3_보온 작업 근접 소음 및 선박 망치질.wav, STOI: 0.7767, PESQ: 1.5971
노이즈제거 전: speech5_보온 작업 근접 소음 및 선박 망치질.wav, STOI: 0.6339, PESQ: 1.0720
노이즈제거 후: speech5_보온 작업 근접 소음 및 선박 망치질.wav, STOI: 0.7554, PESQ: 1.3734
노이즈제거 전: speech3_대형 환기팬 및 설비 소음.wav, STOI: 0.7680, PESQ: 1.1921
노이즈제거 후: speech3_대형 환기팬 및 설비 소음.wav, STOI: 0.8386, PESQ: 1.6425
노이즈제거 전: speech4_스프레이 작업 소음 (고음).wav, STOI: 0.5899, PESQ: 1.0699
노이즈제거 후: speech4_스프레이 작업 소음 (고음).wav, STOI: 0.6671, PESQ: 1.2349
노이즈제거 전: speech5_대형 환기팬 및 설비 소음.wav, STOI: 0.7152, PESQ: 1.1098
노이즈제거 후: speech5_대형 환기팬 및 설비 소음.wav, STOI: 0.8031, PESQ: 1.5098
노이즈제거 전: speech2_초고속 모터 작업 소음.wav, STOI: 0.7756, PESQ: 1.2344
노이즈제거 후: speech2_초고속 모터 작업

In [18]:
import os

if __name__ == "__main__":
    SIG_DIR = "sounds/before_noise"
    NOISY_DIR = "sounds/after_noise"
    NOISE_DIR = "sounds/noise/noise_clip"
    TARGET_DIR = "sounds/after_noise_denoised"
    
    for file in os.listdir(NOISY_DIR):
        if not file.endswith(".wav"):
            continue
        clean_file = os.path.join(SIG_DIR, f'{file.split("_")[0]}.wav')
        noisy_file = os.path.join(NOISY_DIR, file)
        si_sdr_score = measure_si_sdr(clean_file, noisy_file)
        denoised_file = os.path.join(TARGET_DIR, file)
        denoised_si_sdr_score = measure_si_sdr(clean_file, denoised_file)
        print(f"노이즈제거 전: {file}, SI-SDR: {si_sdr_score:.4f}")
        print(f"노이즈제거 후: {file}, SI-SDR: {denoised_si_sdr_score:.4f}")


노이즈제거 전: speech4_단열폼 미장 소음.wav, SI-SDR: -2.9846
노이즈제거 후: speech4_단열폼 미장 소음.wav, SI-SDR: -0.3845
노이즈제거 전: speech4_대형 환기팬 및 설비 소음.wav, SI-SDR: -3.1377
노이즈제거 후: speech4_대형 환기팬 및 설비 소음.wav, SI-SDR: 1.8493
노이즈제거 전: speech3_보온 작업 근접 소음 및 선박 망치질.wav, SI-SDR: -3.0656
노이즈제거 후: speech3_보온 작업 근접 소음 및 선박 망치질.wav, SI-SDR: 7.3190
노이즈제거 전: speech5_보온 작업 근접 소음 및 선박 망치질.wav, SI-SDR: -2.8758
노이즈제거 후: speech5_보온 작업 근접 소음 및 선박 망치질.wav, SI-SDR: 4.2861
노이즈제거 전: speech3_대형 환기팬 및 설비 소음.wav, SI-SDR: -2.9607
노이즈제거 후: speech3_대형 환기팬 및 설비 소음.wav, SI-SDR: 9.0310
노이즈제거 전: speech4_스프레이 작업 소음 (고음).wav, SI-SDR: -3.0067
노이즈제거 후: speech4_스프레이 작업 소음 (고음).wav, SI-SDR: 6.0086
노이즈제거 전: speech5_대형 환기팬 및 설비 소음.wav, SI-SDR: -3.1416
노이즈제거 후: speech5_대형 환기팬 및 설비 소음.wav, SI-SDR: 6.1515
노이즈제거 전: speech2_초고속 모터 작업 소음.wav, SI-SDR: -3.0584
노이즈제거 후: speech2_초고속 모터 작업 소음.wav, SI-SDR: 7.3895
노이즈제거 전: speech3_저속 스프레이 작업 소음 (직접).wav, SI-SDR: -3.1215
노이즈제거 후: speech3_저속 스프레이 작업 소음 (직접).wav, SI-SDR: 7.2390
노이즈제거 전: speech3_스프레이 작업 소음 (저음).w